# What type of weapons are typically used in mass shootings

In [19]:
#Setup
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from scipy.stats import pearsonr, linregress

# Mass Killings Files
mass_kills_df = "./Resources/mass_killing_incidents_public.csv"
mass_weapons_df = "./Resources/mass_killing_weapons_public.csv"

# Load the data to a DataFrame
mass_kills = pd.read_csv(mass_kills_df)
mass_weapons = pd.read_csv(mass_weapons_df)

# Display the DataFrame
#mass_kills.head()
#mass_weapons.head()



In [20]:
#Merge Data
mass_kills_complete = pd.merge(mass_kills, mass_weapons, how="left", 
                               on=["incident_id", "incident_id"])
mass_kills_complete.head()

,incident_id,date,city,state,num_offenders,num_victims_killed,num_victims_injured,firstcod,secondcod,type,situation_type,location_type,location,longitude,latitude,narrative,weapon_id,weapon_type,gun_class,gun_type
0,568,2023-05-06,Allen,TX,1,8,7,Shooting,NaN,Public,Indiscriminate,Commercial/Retail/Entertainment,Commercial/Retail,-96.661856,33.130229,A gunman opened fire at the Allen Premium Outl...,883.0,gun,LG,semiautomatic rifle
1,568,2023-05-06,Allen,TX,1,8,7,Shooting,NaN,Public,Indiscriminate,Commercial/Retail/Entertainment,Commercial/Retail,-96.661856,33.130229,A gunman opened fire at the Allen Premium Outl...,884.0,gun,HG,handgun
2,567,2023-05-02,Lake Wales,FL,1,4,0,Shooting,NaN,Family,NaN,Residence/Other shelter,Residence,-81.584680,27.912660,"Al Stenson, 38, fatally shot his girlfriend an...",882.0,gun,UG,NaN
3,565,2023-04-30,Mojave,CA,1,4,0,Shooting,NaN,Unsolved,NaN,Residence/Other shelter,Residence,-118.177390,35.055360,Three women and one man were fatally shot late...,880.0,gun,UG,NaN
4,566,2023-04-30,Henryetta,OK,1,6,0,Shooting,NaN,Family,NaN,Residence/Other shelter,Residence,-95.936810,35.439920,"Jesse McFadden fatally shot his wife, her chil...",881.0,gun,HG,semiautomatic handgun


## Type of firearm used in mass shootings 

In [22]:
#Select Columns
weapons_info = mass_kills_complete[["incident_id", "date", "city", "state", 
                                   "num_victims_killed", "location_type", 
                                    "gun_type", "weapon_type"]]
#Rename
weapons_df = weapons_info.copy()

weapons_df.head()

,incident_id,date,city,state,num_victims_killed,location_type,gun_type,weapon_type
0,568,2023-05-06,Allen,TX,8,Commercial/Retail/Entertainment,semiautomatic rifle,gun
1,568,2023-05-06,Allen,TX,8,Commercial/Retail/Entertainment,handgun,gun
2,567,2023-05-02,Lake Wales,FL,4,Residence/Other shelter,NaN,gun
3,565,2023-04-30,Mojave,CA,4,Residence/Other shelter,NaN,gun
4,566,2023-04-30,Henryetta,OK,6,Residence/Other shelter,semiautomatic handgun,gun


In [25]:
#Rename the columns
weapons_df = weapons_df.rename(columns={
    "incident_id": "ID",
    "date": "Date",
    "city": "City",
    "state": "State",
    "num_victims_killed": "# of Victims",
    "location_type": "Location Type",
    "gun_type": "Gun Type",
    "weapon_type": "Weapon Type",
})

weapons_df.head()

,ID,Date,City,State,# of Victims,Location Type,Gun Type,Weapon Type
0,568,2023-05-06,Allen,TX,8,Commercial/Retail/Entertainment,semiautomatic rifle,gun
1,568,2023-05-06,Allen,TX,8,Commercial/Retail/Entertainment,handgun,gun
2,567,2023-05-02,Lake Wales,FL,4,Residence/Other shelter,NaN,gun
3,565,2023-04-30,Mojave,CA,4,Residence/Other shelter,NaN,gun
4,566,2023-04-30,Henryetta,OK,6,Residence/Other shelter,semiautomatic handgun,gun
